<a href="https://colab.research.google.com/github/shankar-296/sithafal/blob/main/sitafal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfminer.six
!pip install streamlit
!pip install pickle5
!pip install langchain
!pip install langchain-groq
!pip install faiss-cpu
!pip install huggingface_hub

In [2]:
!pip install -U langchain-community

In [3]:
!pip install pdf2image

In [4]:
!pip install fitz

In [5]:
!pip install pytesseract

In [6]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.6 MB/s eta 0:00:00


In [7]:
!pip install sentence-transformers

# **TASK - 1**

In [ ]:
from pdfminer.high_level import extract_text
import os
import pickle
import time
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from google.colab import files

# Initialize LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.1-70b-versatile"
)

# File upload in Colab
uploaded_files = files.upload()

# Path to save FAISS index
file_path = "faiss_store_openai.pkl"

# Process PDFs
def process_pdfs():
    all_text = ""

    # Extract text from uploaded PDFs
    for uploaded_file in uploaded_files.keys():
        try:
            extracted_text = extract_text(uploaded_file)
            print(f"Extracted text from {uploaded_file}:")
            print(extracted_text[:500])  # Display the first 500 characters for validation
            all_text += extracted_text + "\n"
        except Exception as e:
            print(f"Error extracting text from {uploaded_file}: {e}")
            continue

    if not all_text.strip():
        print("Error: No text extracted from the provided PDFs. Please check the files.")
        raise ValueError("No text to process.")

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_text(all_text)
    print(f"Number of text chunks created: {len(text_chunks)}")

    if not text_chunks:
        print("Error: No text chunks created. Please check the extracted text.")
        raise ValueError("No text chunks to process.")

    # Generate embeddings and create vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    try:
        vectorstore_openai = FAISS.from_texts(text_chunks, embeddings)
    except Exception as e:
        print(f"Error creating FAISS vector store: {e}")
        raise

    # Save FAISS index to a pickle file
    with open(file_path, "wb") as f:
        pickle.dump(vectorstore_openai, f)

    print("Embedding vectors created and FAISS index saved successfully.")

# Run the PDF processing function
process_pdfs()

# Query input
query = input("Ask a Question: ")
if query:
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            vectorstore = pickle.load(f)
            chain = RetrievalQA.from_llm(llm=llm, retriever=vectorstore.as_retriever())

        # Get response
        try:
            result = chain.run(query)
            # Display the answer
            print("Answer:")
            print(result)
        except Exception as e:
            print(f"Error during query handling: {e}")
    else:
        print("Error: FAISS index file not found. Please ensure the processing step completed successfully.")


# **TASK - 2**

In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import pickle
import time
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Initialize LLM
llm = ChatGroq(temperature=0, groq_api_key="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", model_name="llama-3.1-70b-versatile")

file_path = "faiss_store_openai.pkl"

# Function to scrape content from a website
def scrape_website(url):
    # Send a request to the website
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract text from all paragraph tags (you can adjust this as needed)
        paragraphs = soup.find_all('p')
        text = ' '.join([para.get_text() for para in paragraphs])
        return text
    else:
        print(f"Failed to retrieve the webpage: {url}")
        return ""

# Process websites after input
def process_websites(urls):
    all_text = ""

    # Scrape text from all websites
    for url in urls:
        print(f"Processing website: {url}")
        extracted_text = scrape_website(url)
        all_text += extracted_text + "\n"

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_text(all_text)

    # Create embeddings and vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore_openai = FAISS.from_texts(text_chunks, embeddings)

    # Save FAISS index
    print("Embedding Vector Started Building...✅✅✅")
    time.sleep(2)

    # Save the FAISS index to a pickle file
    with open(file_path, "wb") as f:
        pickle.dump(vectorstore_openai, f)

    print("Text extracted and FAISS index saved.")

# User input for URLs
urls = input("Enter the website URLs (comma-separated): ").split(',')

# Run processing after URL input
process_websites(urls)

# Query input
query = input("Ask a Question: ")
if query:
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            vectorstore = pickle.load(f)
            chain = RetrievalQA.from_llm(llm=llm, retriever=vectorstore.as_retriever())

        # Get response
        result = chain.run(query)

        # Display answer
        print("Answer:")
        print(result)
